In [1]:
! pip install --upgrade --quiet langchain langchain-openai langchainhub

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [3]:
from langchain_core.tools import tool


def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base ** exponent


tools = [multiply, add, exponentiate]

In [4]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

In [5]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [6]:
# Choose the LLM that will drive the agent
# Only certain models support this
model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(model, tools, prompt)

In [7]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

KeyError: 'tools'

In [ ]:
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)

In [8]:
import nest_asyncio

nest_asyncio.apply()

In [9]:
from src.eval.envs.env import Env


class MathEnv(Env):
    async def init(self, init_params: dict) -> str:
        pass

    async def run_command(self, command_name: str, command_params: dict) -> int:
        if command_name == 'multiply':
            first_int = command_params['first_int']
            second_int = command_params['second_int']
            return multiply(first_int, second_int)
        elif command_name == 'add':
            first_int = command_params['first_int']
            second_int = command_params['second_int']
            return add(first_int, second_int)
        elif command_name == 'exponentiate':
            base = command_params['base']
            exponent = command_params['exponent']
            return exponentiate(base, exponent)
        else:
            print("Unknown command")

    async def get_tools(self) -> list[dict]:
        return [{
            "type": "function",
            "function": {
                "name": "multiply",
                "description": "Multiply two integers together.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "first_int": {
                            "type": "integer",
                            "description": "First integer",
                        },
                        "second_int": {
                            "type": "integer",
                            "description": "Second integer",
                        },
                    },
                    "required": ["first_int", "second_int"],
                },
            }
        }, {
            "type": "function",
            "function": {
                "name": "add",
                "description": "Add two integers together.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "first_int": {
                            "type": "integer",
                            "description": "First integer",
                        },
                        "second_int": {
                            "type": "integer",
                            "description": "Second integer",
                        },
                    },
                    "required": ["first_int", "second_int"],
                },
            }
        }, {
            "type": "function",
            "function": {
                "name": "exponentiate",
                "description": "Exponentiate the base to the exponent power.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "base": {
                            "type": "integer",
                            "description": "Base integer",
                        },
                        "exponent": {
                            "type": "integer",
                            "description": "Exponent integer",
                        },
                    },
                    "required": ["base", "exponent"],
                },
            }
        }]

    async def get_state(self) -> str:
        pass

In [10]:
from langchain_core.tools import StructuredTool
from src.tools.tools import parse_tool

math_env = MathEnv()
custom_tools = [parse_tool(tool_desc['function'], math_env) for tool_desc in await math_env.get_tools()]

# custom_tools = [StructuredTool.from_function(name="multiply", func=multiply, description="Multiply two integers"),
#                 StructuredTool.from_function(name="add", func=add, description="Add two integers"),
#                 StructuredTool.from_function(name="exponentiate", func=exponentiate,
#                                              description="Exponentiate the base to the exponent power")]

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()
model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)
agent = create_openai_tools_agent(model, custom_tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=custom_tools, verbose=True)

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [11]:
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


<coroutine object parse_tool.<locals>.tool_impl at 0x127c87440>
Invoking: `add` with `{'first_int': 12, 'second_int': 3}`


<coroutine object parse_tool.<locals>.tool_impl at 0x137588d40>
Invoking: `multiply` with `{'first_int': 243, 'second_int': 15}`
responded: I will now calculate 3 to the power of 5 and the sum of 12 and 3 in parallel.

<coroutine object parse_tool.<locals>.tool_impl at 0x1105ec940>
Invoking: `exponentiate` with `{'base': 405, 'exponent': 2}`
responded: The result of 3 to the power of 5 is 243, and the sum of 12 and 3 is 15. Now, I will multiply these two results together.

<coroutine object parse_tool.<locals>.tool_impl at 0x127c87b40>The result of multiplying 243 by 15 is 3645. Now, I will square this result.
The final result of squaring 3645 is 13,297,025.

> Finished chain.


/Users/Maria.Tigina/PycharmProjects/agents-eval/venv/lib/python3.11/site-packages/langchain/chains/base.py:153: RuntimeWarning: coroutine 'parse_tool.<locals>.tool_impl' was never awaited
  self._call(inputs, run_manager=run_manager)


{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result',
 'output': 'The result of multiplying 243 by 15 is 3645. Now, I will square this result.\nThe final result of squaring 3645 is 13,297,025.'}